In [ ]:
# Installation
using Revise
using LinearAlgebra,Statistics,StatsBase,Distributions
using Plots,NamedColors ; theme(:default)
using FFTW
using ProgressMeter
using Random
Random.seed!(0)
using HawkesSimulator; const global H = HawkesSimulator
using NamedColors

Step 1. Comparing Linear Hawkes mean and covariance, with and without weights

In [ ]:
# Initializing variables
# one E, one I population
v_rest = -70
alpha = 1
n = 1
tau = [20.0, 10.0]*1E-3   #time constant, [ E, I ]
w_zero = [0.0 0.0
        0.0 0.0]
w = [1.25 -0.65
     1.2 -0.5]
w_hawkes = w .* (alpha*(1/n))
w_hawkes_zero = [0.0 0.0
                0.0 0.0]
time_step = 0.1*1E-3
tau_noise = 50*1E-3

In [ ]:
function onedmat(x::Real)
  return cat(x;dims=2)
end

In [ ]:
function rate_powerlaw(v::Float64)
    diff = v - v_rest
    diff = (diff < 0) ? 0 : diff
    return alpha*(diff^n)
end

function plot_powerlaw()
    count = 81
    v_arr = zeros(count)
    v_arr[1] = -90
    for i in 2:count
        v_arr[i] = v_arr[i-1] +1
    end
    rates = rate_powerlaw.(v_arr)
    plot(v_arr, rates, xlabel="Voltage (mV)", ylabel="Rate (Hz)", color="black", legend=false, fmt=:png)
end

plot_powerlaw()

In [ ]:
function plot_count(points)
    y = collect(1:length(points))
    plt = plot(xlabel="time (s)", ylabel="count")
    plot!(plt, points, y, label = "N(t)")
end

function plot_count(points_E, points_I)
    y = collect(1:length(points_E))
    plt = plot(xlabel="time (s)", ylabel="count")
    plot!(plt, points_E, y, label = "N_E(t)")
    y = collect(1:length(points_I))
    plot!(plt, points_I, y, label = "N_I(t)")
end

function plot_count(points_E, points_I, remean, rimean, count)
    y = collect(1:count)
    y1 = zeros(count)
    y2 = zeros(count)
    for i in 1:count
        y1[i] = remean*points_E[i]
        y2[i] = rimean*points_I[i]
    end
    plt = plot(xlabel="time (s)", ylabel="Spike Count", legend=:bottomright, fmt=:png)
    plot!(plt, points_E[1:count], y, label = "N_E(t)", color="blue")
    plot!(plt, points_I[1:count], y, label = "N_I(t)", color="red")
    plot!(plt, points_E[1:count],y1, label="E[N_E(t)]", color="dark blue")
    plot!(plt, points_I[1:count],y2, label="E[N_I(t)]", color="dark red")
end

In [ ]:
function simulate_hawkes!(networks, num_spikes)
    t_now = 0.0
    H.reset!.(networks) # clear spike trains etc
    for k in 1:num_spikes
        t_now = H.dynamics_step!(t_now, networks)
        if k%1_000 == 0
            H.clear_trains!(networks[1].postpops)
            H.clear_trains!(networks[2].postpops)
        end # clearing trains after every 1000 spikes
    end
    return t_now
end

In [ ]:
function run_hawkes_multiple_h!(num_h, h_arr, w_hawkes, rates_hawkes, pop_E, pop_I, covE_h, covI_h, covEI_h)
    n_spikes = 500_000
    @showprogress 1.0 "Running Hawkes multiple h..." for i in 1:num_h
        baseline_rate_e = [alpha*(h_arr[i])]
        baseline_rate_i = [alpha*(h_arr[i])]
        popstate_E = H.PopulationState(pop_E,baseline_rate_e)
        popstate_I = H.PopulationState(pop_I, baseline_rate_i)
        network_E = H.InputNetwork(popstate_E,[popstate_E, popstate_I],[onedmat(w_hawkes[1,1]),onedmat(w_hawkes[1,2])])
        network_I = H.InputNetwork(popstate_I,[popstate_E, popstate_I],[onedmat(w_hawkes[2,1]),onedmat(w_hawkes[2,2])])
        simulate_hawkes!([network_E, network_I],n_spikes)
        rates_hawkes[i,:] = [H.numerical_rate(popstate_E.trains_history[1][2000:end]), H.numerical_rate(popstate_I.trains_history[1][2000:end])]
        spikesE = H.bin_spikes(popstate_E.trains_history[1], 1.0, max(popstate_E.trains_history[1][end], popstate_I.trains_history[1][end]))
        spikesI = H.bin_spikes(popstate_I.trains_history[1], 1.0, max(popstate_E.trains_history[1][end], popstate_I.trains_history[1][end]))
        binrate_hawkes = zeros(2, Int(floor(sizeof(spikesE)/sizeof(spikesE[1]))))
        binrate_hawkes[1,:] = spikesE ./ 1
        binrate_hawkes[2,:] = spikesI ./ 1
        covariance = cov(binrate_hawkes; dims=2)
        covE_h[i] = covariance[1,1]
        covI_h[i] = covariance[2,2]
        covEI_h[i] = covariance[1,2]
    end
end

In [ ]:
h_max = 20.0
h_min = 1.0
step = 1
num_h = Int(ceil((h_max-h_min+1)/step))
h_arr = zeros(num_h)
h_arr[1] = h_min
rates_hawkes = zeros(num_h,2)
covE_h = zeros(num_h)
covI_h = zeros(num_h)
covEI_h = zeros(num_h)
rates_hawkes_now =zeros(num_h,2)
covE_h_now = zeros(num_h)
covI_h_now = zeros(num_h)
covEI_h_now = zeros(num_h)

function compare_hawkes()
    for i in 2:num_h
        h_arr[i] = h_arr[i-1] + step
    end
    
    tau_E = 10.0
    tau_I = 5.0
    pop_E = H.PopulationExp(tau_E, H.NLIdentity())
    pop_I = H.PopulationExp(tau_I, H.NLIdentity())
    run_hawkes_multiple_h!(num_h, h_arr, w_hawkes, rates_hawkes, pop_E, pop_I, covE_h, covI_h, covEI_h)
    run_hawkes_multiple_h!(num_h, h_arr, w_hawkes_zero, rates_hawkes_now, pop_E, pop_I, covE_h_now, covI_h_now, covEI_h_now)
    
end

In [ ]:
compare_hawkes()

In [ ]:
plt1 = plot(xlabel="Linear Hawkes mean E (Hz)", ylabel="covariance EE", fmt = :png, legend=:bottomright)
plot!(plt1, rates_hawkes[:,1] , covE_h[:], label = "weights", color="blue")
plot!(plt1, rates_hawkes_now[:,1] , covE_h_now[:], label = "no weights", color="purple")
plot!(plt1, rates_hawkes[:,1] , rates_hawkes[:,1], label = "y = x", color="black", linestyle=:dot)

plt2 = plot(xlabel="Linear Hawkes mean I (Hz)", ylabel="covariance II", fmt = :png, legend=:bottomright)
plot!(plt2, rates_hawkes[:,2] , covI_h[:], label = "weights", color="red")
plot!(plt2, rates_hawkes_now[:,2] , covI_h_now[:], label = "no weights", color="purple")
plot!(plt2, rates_hawkes[:,2] , rates_hawkes[:,2], label = "y = x", color="black", linestyle=:dot)

plt3 = plot(xlabel="Linear Hawkes h (mV)", ylabel="covariance EI", fmt = :png, legend=:topleft)
plot!(plt3, h_arr, covEI_h[:], label = "weights", color="green")
plot!(plt3, h_arr, covEI_h_now[:], label = "no weights", color="purple")

plot!(plt1, plt2, plt3)

In [ ]:
plt4 = plot(xlabel="Linear Hawkes h (mV) weights", ylabel="E neuron", fmt = :png, legend=:bottomright)
plot!(plt4, h_arr, [rates_hawkes[:,1] covE_h[:]], label = ["mean E (Hz)" "covariance EE"], color=["blue" "purple"])

plt5 = plot(xlabel="Linear Hawkes h (mV) weights", ylabel="I neuron", fmt = :png, legend=:bottomright)
plot!(plt5, h_arr, [rates_hawkes[:,2] covI_h[:]], label = ["mean I (Hz)" "covariance II"], color=["red" "purple"])

plt6 = plot(xlabel="Linear Hawkes h (mV) no weights", ylabel="E neuron", fmt = :png, legend=:bottomright)
plot!(plt6, h_arr, [rates_hawkes_now[:,1] covE_h_now[:]], label = ["mean E (Hz)" "covariance EE"], color=["blue" "purple"])

plt7 = plot(xlabel="Linear Hawkes h (mV) no weights", ylabel="I neuron", fmt = :png, legend=:bottomright)
plot!(plt7, h_arr, [rates_hawkes_now[:,2] covI_h_now[:]], label = ["mean I (Hz)" "covariance II"], color=["red" "purple"])


plot!(plt6, plt7, plt4, plt5)

Result : Hawkes with no weights is a Poisson process, since mean = variance

on adding weights, mean and variance increase. variance > mean

Step 2. Comparing linear Hawkes mean & covariance & Analytic mean and covariance of the linear rate model, without weights

In [ ]:
function single_h_linear_hawkes_simulation(h::Real, w_hawkes::Matrix{Float64})
    n_spikes = 500_000
    baseline_rate_e = [(alpha^(1/n))*h]
    baseline_rate_i = [(alpha^(1/n))*h]
    tau_E = 10.0
    tau_I = 5.0
    pop_E = H.PopulationExp(tau_E, H.NLIdentity())
    pop_I = H.PopulationExp(tau_I, H.NLIdentity())
    popstate_E = H.PopulationState(pop_E,baseline_rate_e)
    popstate_I = H.PopulationState(pop_I, baseline_rate_i)
    network_E = H.InputNetwork(popstate_E,[popstate_E, popstate_I],[onedmat(w_hawkes[1,1]),onedmat(w_hawkes[1,2])])
    network_I = H.InputNetwork(popstate_I,[popstate_E, popstate_I],[onedmat(w_hawkes[2,1]),onedmat(w_hawkes[2,2])])
    simulate_hawkes!([network_E, network_I],n_spikes)
    rate = [H.numerical_rate(popstate_E.trains_history[1][10000:end]), H.numerical_rate(popstate_I.trains_history[1][10000:end])]
    spikesE = H.bin_spikes(popstate_E.trains_history[1], 1.0, max(popstate_E.trains_history[1][end], popstate_I.trains_history[1][end]))
    spikesI = H.bin_spikes(popstate_I.trains_history[1], 1.0, max(popstate_E.trains_history[1][end], popstate_I.trains_history[1][end]))
    binrate_hawkes = zeros(2, Int(floor(sizeof(spikesE)/sizeof(spikesE[1]))))
    binrate_hawkes[1,:] = spikesE ./ 1
    binrate_hawkes[2,:] = spikesI ./ 1
    covariance = cov(binrate_hawkes; dims=2)
    return rate, covariance, popstate_E.trains_history[1], popstate_I.trains_history[1], network_E, network_I
end

In [ ]:
# dx/dt = Ax + h + noise
# E[x] = -h * A^-1
# Cov = lyap(A, Cov(noise))

function run_analytic_linear_single_h(time_step::Real, h::Vector{Float64}, alpha::Real, W::Matrix{Float64}, cov_noise::Matrix{Float64})
    
#     cov_noise == cov_wiener
    cov_noise[1,:] = cov_noise[1,:] .* (alpha/tau[1])
    cov_noise[2,:] = cov_noise[2,:] .* (alpha/tau[2])
    h[1] = h[1]/tau[1]
    h[2] = h[2]/tau[2]

    A = [(alpha*W[1,1]-1)/tau[1] alpha*W[1,2]/tau[1]
        alpha*W[2,1]/tau[2] (alpha*W[2,2]-1)/tau[2]]
    rate = -A\h
    covariance = lyap(A, cov_noise)
    return rate,covariance
end

In [ ]:
h = 5.0
baseline_rate = [alpha*h
                alpha*h]
cov_noise = [2*alpha*h 0.0
            0.0 2*alpha*h]
rate_linear,covariance_linear = run_analytic_linear_single_h(time_step, baseline_rate, alpha, w_zero, cov_noise)
println(rate_linear)
println(covariance_linear)

In [ ]:
rate_hawkes, covariance_hawkes, train_hawkes_E, train_hawkes_I, network_E, network_I = single_h_linear_hawkes_simulation(h, w_hawkes_zero)
println(rate_hawkes)
println(covariance_hawkes)

In [ ]:
function run_analytic_linear_multiple_h!(num_h, h_arr, w, rates_linear, covE_l, covI_l, covEI_l)
    @showprogress 1.0 "Running Analytic linear multiple h..." for i in 1:num_h
        cov_noise = [2*alpha*h_arr[i] 0.0
                    0.0 2*alpha*h_arr[i]]
        baseline_rate = [alpha*h_arr[i]; alpha*h_arr[i]]
        
        rates_linear[i,:], covariance = run_analytic_linear_single_h(time_step, baseline_rate, alpha, w, cov_noise)
        covE_l[i] = covariance[1,1]
        covI_l[i] = covariance[2,2]
        covEI_l[i] = covariance[1,2]
    end
end

In [ ]:
h_max = 20.0
h_min = 1.0
step = 1
num_h = Int(ceil((h_max-h_min+1)/step))
h_arr = zeros(num_h)
h_arr[1] = h_min

rates_linear_now = zeros(num_h,2)
covE_l_now = zeros(num_h)
covI_l_now = zeros(num_h)
covEI_l_now = zeros(num_h)

rates_hawkes_now = zeros(num_h,2)
covE_h_now = zeros(num_h)
covI_h_now = zeros(num_h)
covEI_h_now = zeros(num_h)

function compare_hawkes_and_linear_now()
    for i in 2:num_h
        h_arr[i] = h_arr[i-1] + step
    end
    
    run_analytic_linear_multiple_h!(num_h, h_arr, w_zero, rates_linear_now, covE_l_now, covI_l_now, covEI_l_now)
    
    tau_E = 10.0
    tau_I = 5.0
    pop_E = H.PopulationExp(tau_E, H.NLIdentity())
    pop_I = H.PopulationExp(tau_I, H.NLIdentity())
    run_hawkes_multiple_h!(num_h, h_arr, w_hawkes_zero, rates_hawkes_now, pop_E, pop_I, covE_h_now, covI_h_now, covEI_h_now)
    
end

In [ ]:
compare_hawkes_and_linear_now()

In [ ]:
plt8 = plot(xlabel="mean E (Hz)", ylabel="covariance EE", fmt = :png, legend=:bottomright)
plot!(plt8, rates_linear_now[:,1] , covE_l_now[:], label = "rate model", color="blue")
plot!(plt8, rates_hawkes_now[:,1] , covE_h_now[:], label = "hawkes", color="purple")
plot!(plt8, rates_linear_now[:,1] , rates_linear_now[:,1], label = "y = x", color="black", linestyle=:dot)

plt9 = plot(xlabel="mean I (Hz)", ylabel="covariance II", fmt = :png, legend=:bottomright)
plot!(plt9, rates_linear_now[:,2] , covI_l_now[:], label = "rate model", color="blue")
plot!(plt9, rates_hawkes_now[:,2] , covI_h_now[:], label = "hawkes", color="purple")
plot!(plt9, rates_linear_now[:,2] , rates_linear_now[:,2], label = "y = x", color="black", linestyle=:dot)

plt10 = plot(xlabel="h (mV)", ylabel="covariance EI", fmt = :png, legend=:bottomleft)
plot!(plt10, h_arr, covEI_l_now[:], label = "rate model", color="green")
plot!(plt10, h_arr, covEI_h_now[:], label = "hawkes", color="purple")

plot!(plt8, plt9, plt10)
# Comparing linear Hawkes with analytic mean and covariance of linear rate model, NO WEIGHTS

Comparison with weights

In [ ]:
# h_max = 20.0
# h_min = 1.0
# step = 1
# num_h = Int(ceil((h_max-h_min+1)/step))
# h_arr = zeros(num_h)
# h_arr[1] = h_min
rates_linear = zeros(num_h,2)
covE_l = zeros(num_h)
covI_l = zeros(num_h)
covEI_l = zeros(num_h)

# rates_hawkes = zeros(num_h,2)
# covE_h = zeros(num_h)
# covI_h = zeros(num_h)
# covEI_h = zeros(num_h)

function compare_hawkes_and_linear()
#     for i in 2:num_h
#         h_arr[i] = h_arr[i-1] + step
#     end
    
    run_analytic_linear_multiple_h!(num_h, h_arr, w, rates_linear, covE_l, covI_l, covEI_l)
    
#     tau_E = 10.0
#     tau_I = 5.0
#     pop_E = H.PopulationExp(tau_E, H.NLIdentity())
#     pop_I = H.PopulationExp(tau_I, H.NLIdentity())
#     run_hawkes_multiple_h!(num_h, h_arr, w_hawkes, rates_hawkes, pop_E, pop_I, covE_h, covI_h, covEI_h)
    
end

In [ ]:
compare_hawkes_and_linear()

In [ ]:
plt = plot(xlabel="h (mV)", ylabel="covariance EE", fmt = :png, legend=:bottomright)
plot!(plt, h_arr , covE_l[:], label = "rate model", color="blue")
plot!(plt, h_arr , covE_h[:], label = "hawkes", color="purple")

In [ ]:
plt = plot(xlabel="h (mV)", ylabel="covariance II", fmt = :png, legend=:bottomright)
plot!(plt, h_arr , covI_l[:], label = "rate model", color="blue")
plot!(plt, h_arr , covI_h[:], label = "hawkes", color="purple")

In [ ]:
plt = plot(xlabel="h (mV)", ylabel="covariance II", fmt = :png, legend=:bottomright)
plot!(plt, h_arr , covI_l[:], label = "rate model", color="blue")
plot!(plt, h_arr , covI_h[:], label = "hawkes", color="purple")

In [ ]:
plt = plot(xlabel="h (mV)", ylabel="covariance EE", fmt = :png, legend=:bottomright)
plot!(plt, h_arr , covE_l[:], label = "rate model", color="blue")
plot!(plt, h_arr , covE_h[:], label = "hawkes", color="purple")

In [ ]:
plt = plot(xlabel="h (mV)", ylabel="covariance EI", fmt = :png, legend=:topleft)
plot!(plt, h_arr, covEI_l[:], label = "rate model", color="green")
plot!(plt, h_arr, covEI_h[:], label = "hawkes", color="purple")

In [ ]:
plt = plot(xlabel="mean E (Hz)", ylabel="covariance EE", fmt = :png, legend=:bottomright)
plot!(plt, rates_linear[:,1] , covE_l[:], label = "rate model", color="blue")
plot!(plt, rates_hawkes[:,1] , covE_h[:], label = "hawkes", color="purple")
plot!(plt, rates_linear[:,1] , rates_linear[:,1], label = "y = x", color="black", linestyle=:dot)

In [ ]:
plt = plot(xlabel="mean I (Hz)", ylabel="covariance II", fmt = :png, legend=:bottomright)
plot!(plt, rates_linear[:,2] , covI_l[:], label = "rate model", color="blue")
plot!(plt, rates_hawkes[:,2] , covI_h[:], label = "hawkes", color="purple")
plot!(plt, rates_linear[:,2] , rates_linear[:,2], label = "y = x", color="black", linestyle=:dot)

In [ ]:
plt = plot(xlabel="mean E (Hz)", ylabel="covariance EE", fmt = :png, legend=:bottomright)
plot!(plt, rates_linear[:,1] , covE_l[:], label = "rate model", color="blue")
plot!(plt, rates_hawkes[:,1] , covE_h[:], label = "hawkes", color="purple")
plot!(plt, rates_linear[:,1] , rates_linear[:,1], label = "y = x", color="black", linestyle=:dot)
# without the 2 factor

In [ ]:
plt = plot(xlabel="mean I (Hz)", ylabel="covariance II", fmt = :png, legend=:bottomright)
plot!(plt, rates_linear[:,2] , covI_l[:], label = "rate model", color="blue")
plot!(plt, rates_hawkes[:,2] , covI_h[:], label = "hawkes", color="purple")
plot!(plt, rates_linear[:,2] , rates_linear[:,2], label = "y = x", color="black", linestyle=:dot)

To compare with data from an actual simulation :

In [ ]:
function spiking(probability)
    return rand(Float64) < probability
end

In [ ]:
function dnoise!(noise::Vector{Float64}, wiener::Vector{Float64}, cov_noise::Matrix{Float64})
    wiener = wiener + sqrt(time_step)*[rand(Normal(0,0.5)),rand(Normal(0,0.5))]
    dn = - noise .* time_step + sqrt(2*tau_noise*cov_noise)*wiener
    return dn ./ tau_noise
end

In [ ]:
# Euler Method
function dv_ssn(v::Float64, h_i::Float64, noise_i::Float64, tau_i::Float64, w_arr::Vector{Float64}, rate_arr::Vector{Float64})
    num_neurons = size(w_arr, 1)
    dv = v_rest - v + h_i + noise_i
    for i in 1:num_neurons
        dv += w_arr[i]*rate_arr[i]
    end
    return dv*time_step/tau_i
end

In [ ]:
function simulate_ssn!(h, num_steps, t_arr, v_excite, v_inhibit, rate, spike_trainE, spike_trainI, cov_noise)
    
    v_excite[1] = v_rest
    v_inhibit[1] = v_rest
    rate[1,:] = [rate_powerlaw(v_excite[1]), rate_powerlaw(v_inhibit[1])]
    noise = zeros(2)
    wiener = zeros(2)
    @showprogress 1.0 "Running SSN..." for i in 2:num_steps
        t_arr[i] = t_arr[i-1] + time_step
        noise[:] = noise[:] .+ dnoise!(noise, wiener, cov_noise)
        v_excite[i] = v_excite[i-1] + dv_ssn(v_excite[i-1], h, noise[1], tau[1], [w[1,1], w[1,2]], rate[i-1,:])
        v_inhibit[i] = v_inhibit[i-1] + dv_ssn(v_inhibit[i-1], h, noise[2], tau[2], [w[2,1], w[2,2]], rate[i-1,:])
        rate[i,:] = [rate_powerlaw(v_excite[i]), rate_powerlaw(v_inhibit[i])]
        
        spike_probability = rate[i,:] .* (time_step)
        if spiking(spike_probability[1])
            push!(spike_trainE, t_arr[i])
        end
        if spiking(spike_probability[2])
            push!(spike_trainI, t_arr[i])
        end
    end
    
end

Linear rate model vs Linear Hawkes for a single input current value ( sanity check )

In [ ]:
function single_h_linear_rate_simulation()
    h = 5.0
    t_max = 10000
    num_steps = Int(ceil(t_max/time_step))
    t_arr = zeros(num_steps)
    v_excite = zeros(num_steps)
    v_inhibit = zeros(num_steps)
    spike_trainE = Vector{Float64}()
    spike_trainI = Vector{Float64}()
    rate = zeros(num_steps, 2)
    cov_noise = [alpha*h 0.0
                0.0 alpha*h]
#     cov_noise = [0.0 0.0
#                 0.0 0.0]
    simulate_ssn!(h, num_steps, t_arr, v_excite, v_inhibit, rate, spike_trainE, spike_trainI, cov_noise)
    return t_arr, v_excite, v_inhibit, rate, spike_trainE, spike_trainI
end

In [ ]:
t_arr, v_excite, v_inhibit, rate_linear, spike_trainE, spike_trainI = single_h_linear_rate_simulation()
println([size(spike_trainE), size(spike_trainI)])

In [ ]:
println([mean(v_excite), mean(v_inhibit)])
println([mean(rate_linear[:,1]), mean(rate_linear[:,2])])
# no noise

In [ ]:
println([mean(v_excite), mean(v_inhibit)])
println([mean(rate_linear[:,1]), mean(rate_linear[:,2])])

In [ ]:
println([mean(v_excite), mean(v_inhibit)])
println([mean(rate_linear[:,1]), mean(rate_linear[:,2])])

In [ ]:
rate_hawkes,train_hawkes_E, train_hawkes_I, network_E, network_I = single_h_linear_hawkes_simulation()
print([size(train_hawkes_E), size(train_hawkes_I)])

In [ ]:
length = size(t_arr,1)

In [ ]:
length = size(t_arr,1)
length = 5000
plt = plot(xlabel="time (s)", ylabel="Voltage (mV)", fmt = :png, legend=:bottomright)
plot!(plt, t_arr[1:length] , v_excite[1:length], label = "V_Excite", color="blue")
plot!(plt, t_arr[1:length] , v_inhibit[1:length], label = "V_Inhibit", color="red")

In [ ]:
length = size(t_arr,1)
length = 10000
plt = plot(xlabel="time (s)", ylabel="E Rate (Hz)", fmt = :png, legend=:bottomright)
plot!(plt, t_arr[1:length] , rate_linear[1:length,1], label = "R_Linear", color="blue")
r_excite_hawkes = zeros(length)
r_inhibit_hawkes = zeros(length)
for i in 1:length
    r_excite_hawkes[i] = rate_hawkes[1]
    r_inhibit_hawkes[i] = rate_hawkes[2]
end
plot!(plt, t_arr[1:length], r_excite_hawkes, label = "R_Hawkes",linestyle=:dash,linewidth=2, color="dark blue")

In [ ]:
length = size(t_arr,1)
length = 10000
plt = plot(xlabel="time (s)", ylabel="I Rate (Hz)", fmt = :png, legend=:bottomright)
plot!(plt, t_arr[1:length] , rate_linear[1:length,2], label = "R_Linear",color="red")
plot!(plt, t_arr[1:length], r_inhibit_hawkes, label = "R_Hawkes",linestyle=:dash,linewidth=2, color="dark red")

In [ ]:
spikesE_linear = H.bin_spikes(spike_trainE, 1.0, spike_trainE[end])
spikesI_linear = H.bin_spikes(spike_trainI, 1.0, spike_trainI[end])
println([size(spikesE_linear), size(spikesI_linear)])
binrate_linear = zeros(2, size(spikesI_linear,1))
binrate_linear[1,:] = spikesE_linear ./ 1
binrate_linear[2,:] = spikesI_linear ./ 1
covariance_linear = cov(binrate_linear; dims=2)

println([v_excite[end], v_inhibit[end]])
println([mean(v_excite[:]), mean(v_inhibit[:])])
println([mean(rate_linear[:,1]), mean(rate_linear[:,2])])
println([mean(binrate_linear[1,:]), mean(binrate_linear[2,:])])
print(covariance_linear)

In [ ]:
spikesE = H.bin_spikes(train_hawkes_E, 1.0, train_hawkes_E[end])
spikesI = H.bin_spikes(train_hawkes_I, 1.0, train_hawkes_I[end])
# println([size(spikesE), size(spikesI)])
binrate_hawkes = zeros(2, min(size(spikesE,1), size(spikesI,1)))
binrate_hawkes[1,:] = spikesE[1:size(binrate_hawkes,2)] ./ 1
binrate_hawkes[2,:] = spikesI[1:size(binrate_hawkes,2)] ./ 1
covariance = cov(binrate_hawkes; dims=2)

println(rate_hawkes)
println([mean(binrate_hawkes[1,:]), mean(binrate_hawkes[2,:])])
print(covariance)
# weights

In [ ]:
spikesE_linear = H.bin_spikes(spike_trainE, 1.0, spike_trainE[end])
spikesI_linear = H.bin_spikes(spike_trainI, 1.0, spike_trainI[end])
println([size(spikesE_linear), size(spikesI_linear)])
# binrate_linear = zeros(2, min(size(spikesE_linear,1),size(spikesI_linear,1)))
# binrate_linear[1,:] = spikesE_linear[1:size(binrate_linear,2)] ./ 1
# binrate_linear[2,:] = spikesI_linear[1:size(binrate_linear,2)] ./ 1

binrate_linear = zeros(2, size(spikesE_linear,1))
binrate_linear[1,:] = spikesE_linear ./ 1
binrate_linear[2,:] = spikesI_linear ./ 1
covariance_linear = cov(binrate_linear; dims=2)

println([v_excite[end], v_inhibit[end]])
println(rate_linear[end,:])
println([mean(binrate_linear[1,:]), mean(binrate_linear[2,:])])
print(covariance_linear)

# no weights

In [ ]:
println(train_hawkes_E[20:40])

In [ ]:
spikesE = H.bin_spikes(train_hawkes_E, 1.0, train_hawkes_E[end])
spikesI = H.bin_spikes(train_hawkes_I, 1.0, train_hawkes_I[end])
# println([size(spikesE), size(spikesI)])
binrate_hawkes = zeros(2, size(spikesE,1))
binrate_hawkes[1,:] = spikesE ./ 1
binrate_hawkes[2,:] = spikesI ./ 1
covariance = cov(binrate_hawkes; dims=2)

println(rate_hawkes)
println([mean(binrate_hawkes[1,:]), mean(binrate_hawkes[2,:])])
print(covariance)

# no weights

In [ ]:
var_try = binrate_hawkes[1,:] .- rate_hawkes[1]
var_try = var_try .^2
# println(size(var_try))
variance = sum(var_try)/(size(var_try,1)-1)

In [ ]:
varm(spikesE, mean(spikesE); corrected=true)

In [ ]:
size(spike_trainE,1)

In [ ]:
remean = mean(rate_linear[:,1])
rimean = mean(rate_linear[:,2])
plot_count(spike_trainE, spike_trainI, remean, rimean, 100)
# println(size(rate_linear))

In [ ]:
function rasterplot(tlims = (80.,90.) )
  _trainE = spike_trainE
  plt=plot()
  trainE = filter(t-> tlims[1]< t < tlims[2],_trainE)
  nspk = size(trainE,1)
  scatter!(plt,trainE,fill(2,nspk),markersize=35, markercolor=:blue,markershape=:vline,legend=:topright, label="Excitatory")
  _trainI = spike_trainI
    trainI = filter(t-> tlims[1]< t < tlims[2],_trainI)
  nspk = size(trainI,1)
  scatter!(plt,trainI,fill(1,nspk),markersize=35, markercolor=:red,markershape=:vline,legend=:topright, label="Inhibitory")
  plot!(plt,ylims=(0,3),xlabel="time (s)",fmt=:png)
end

# rasterplot()

In [ ]:
println([v_excite[end], v_inhibit[end]])
# println([vemean, vimean])

println([rate_linear[end,1], rate_linear[end,2]])
# println([remean, rimean])

In [ ]:
length = size(t_arr,1)
# plt = plot(xlabel="time (s)", ylabel="Voltage (mV)", fmt = :png, legend=:bottomright)
# plot!(plt, t_arr[1:length] , v_excite[1:length], label = "V_Excite", color="blue")
# plot!(plt, t_arr[1:length] , v_inhibit[1:length], label = "V_Inhibit", color="red")
# v_excite_mean = zeros(length)
# v_inhibit_mean = zeros(length)
# vemean = mean(v_excite)
# vimean = mean(v_inhibit)
# for i in 1:length
#     v_excite_mean[i] = vemean
#     v_inhibit_mean[i] = vimean
# end
# plot!(plt, t_arr[1:length], v_excite_mean, label = "E[V_Excite]",linestyle=:dash,linewidth=2, color="dark blue")
# plot!(plt, t_arr[1:length], v_inhibit_mean, label = "E[V_Inhibit]",linestyle=:dash,linewidth=2, color="dark red")
plt = plot(xlabel="time (s)", ylabel="Rate (Hz)", fmt = :png, legend=:bottomright)
plot!(plt, t_arr[1:length] , rate_linear[1:length,1], label = "R_Excite", color="blue")
plot!(plt, t_arr[1:length] , rate_linear[1:length,2], label = "R_Inhibit",color="red")
# r_excite_mean = zeros(length)
# r_inhibit_mean = zeros(length)
# remean = mean(rate_linear[:,1])
# rimean = mean(rate_linear[:,2])
# for i in 1:length
#     r_excite_mean[i] = remean
#     r_inhibit_mean[i] = rimean
# end
# plot!(plt, t_arr[1:length], r_excite_mean, label = "E[R_Excite]",linestyle=:dash,linewidth=2, color="dark blue")
# plot!(plt, t_arr[1:length], r_inhibit_mean, label = "E[R_Inhibit]",linestyle=:dash,linewidth=2, color="dark red")

In [ ]:
plt = plot(xlabel="time (s)", ylabel="V (mV)", title = "Linear Rate Model", fmt = :png)
plot!(plt, t_arr[1:20000] , v_excite[1:20000], label = "V_Excite", legend=:bottomright)
plot!(plt, t_arr[1:20000] , v_inhibit[1:20000], label = "V_Inhibit", legend=:bottomright)

In [ ]:
rate_hawkes_arrE = zeros(20000)
rate_hawkes_arrI = zeros(20000)
for i in 1:20000
    rate_hawkes_arrE[i] = rate_hawkes[1]
    rate_hawkes_arrI[i] = rate_hawkes[2]
end
plt = plot(xlabel="time (s)", ylabel="Rate (Hz)", title="Linear Rate Model vs Linear Hawkes", fmt = :png, legend=:bottomright)
plot!(plt, t_arr[1:20000] , rate_linear[1:20000,1], label = "rateE_linear", color = col_exc2)
plot!(plt, t_arr[1:20000], rate_hawkes_arrE, label = "rateE_hawkes", color = col_exc,linestyle=:dash,linewidth=3)
plot!(plt, t_arr[1:20000] , rate_linear[1:20000,2], label = "rateI_linear", color = col_inh2)
plot!(plt, t_arr[1:20000], rate_hawkes_arrI, label = "rateI_hawkes", color = col_inh,linestyle=:dash,linewidth=3)

For multiple h values

In [ ]:
function run_ssn_multiple_h!(num_h, h_arr, rates_ssn, covE_l, covI_l, covEI_l, v_ssn)
    t_max = 2000
    num_steps = Int(ceil(t_max/time_step))
    t_arr = zeros(num_steps)
    v_excite = zeros(num_steps)
    v_inhibit = zeros(num_steps)
    rate = zeros(num_steps, 2)
    @showprogress 1.0 "Running SSN multiple h..." for i in 1:num_h
        spike_trainE = Vector{Float64}()
        spike_trainI = Vector{Float64}()
        cov_noise = [alpha*h_arr[i] 0.0
                    0.0 alpha*h_arr[i]]
#         cov_noise = [0.0 0.0
#                     0.0 0.0]
        simulate_ssn!(h_arr[i], num_steps, t_arr, v_excite, v_inhibit, rate, spike_trainE, spike_trainI, cov_noise)
#         rates_ssn[i,:] = rate[end,:]
        v_ssn[i,1] = v_excite[end]
        v_ssn[i,2] = v_inhibit[end]
        spikesE_linear = H.bin_spikes(spike_trainE, 1.0, max(spike_trainE[end], spike_trainI[end]))
        spikesI_linear = H.bin_spikes(spike_trainI, 1.0, max(spike_trainE[end], spike_trainI[end]))
        binrate_linear = zeros(2, Int(floor(sizeof(spikesE_linear)/sizeof(spikesE_linear[1]))))
        binrate_linear[1,:] = spikesE_linear ./ 1
        binrate_linear[2,:] = spikesI_linear ./ 1 
        rates_ssn[i,1] = mean(binrate_linear[1,:])
        rates_ssn[i,2] = mean(binrate_linear[2,:])
        covariance_linear = cov(binrate_linear; dims=2)
        covE_l[i] = covariance_linear[1,1]
        covI_l[i] = covariance_linear[2,2]
        covEI_l[i] = covariance_linear[1,2]
    end
end

In [ ]:
h_max = 20.0
h_min = 1.0
step = 1
num_h = Int(ceil((h_max-h_min+1)/step))
h_arr = zeros(num_h)
h_arr[1] = h_min
rates_ssn = zeros(num_h,2)
v_ssn = zeros(num_h, 2)
rates_hawkes_linear = zeros(num_h,2)
covE_h = zeros(num_h)
covI_h = zeros(num_h)
covEI_h = zeros(num_h)
covE_l = zeros(num_h)
covI_l = zeros(num_h)
covEI_l = zeros(num_h)

# cov_noise = [sigma_E^2 0.0
#             0.0 sigma_I^2]

function compare()
    for i in 2:num_h
        h_arr[i] = h_arr[i-1] + step
    end
    
    tau_E = 10.0
    tau_I = 5.0
    pop_E = H.PopulationExp(tau_E, H.NLIdentity())
    pop_I = H.PopulationExp(tau_I, H.NLIdentity())
    run_hawkes_multiple_h!(num_h, h_arr, rates_hawkes_linear, pop_E, pop_I, covE_h, covI_h, covEI_h)
    run_ssn_multiple_h!(num_h, h_arr, rates_ssn, covE_l, covI_l, covEI_l, v_ssn)
    
end

In [ ]:
run_ssn_multiple_h!(num_h, h_arr, rates_ssn, covE_l, covI_l, covEI_l, v_ssn)

In [ ]:
compare()

In [ ]:
fano_linear = zeros(length,2)
fano_hawkes = zeros(length,2)
for i in 1:length
    fano_linear[i,1] = covE_l_weights_covdiag[i]/rates_ssn[i,1]
    fano_linear[i,2] = covI_l_weights_covdiag[i]/rates_ssn[i,2]
    fano_hawkes[i,1] = covE_h_weights[i]/rates_hawkes_linear[i,1]
    fano_hawkes[i,2] = covI_h_weights[i]/rates_hawkes_linear[i,2]
end
plt = plot(xlabel="h (mV)", ylabel="Fano Factor = covEE/meanE", fmt = :png, legend=:bottomright)
plot!(plt, h_arr[1:length] , fano_linear[1:length,1], label = "Linear", color="blue")
plot!(plt, h_arr[1:length] , fano_hawkes[1:length,1], label = "Hawkes", color="dark blue", linestyle=:dash)

In [ ]:
plt = plot(xlabel="h (mV)", ylabel="Fano Factor = covII/meanI", fmt = :png, legend=:bottomright)
plot!(plt, h_arr[1:length] , fano_linear[1:length,2], label = "Linear", color="red")
plot!(plt, h_arr[1:length] , fano_hawkes[1:length,2], label = "Hawkes", color="dark red", linestyle=:dash)

In [ ]:
y_e = zeros(num_h)
y_i = zeros(num_h)
for i in 2:num_h
    y_e[i] = 2*abs(fano_hawkes[i,1] - fano_linear[i,1])/(fano_hawkes[i,1] + fano_linear[i,1])
    y_i[i] = abs(fano_hawkes[i,2] - fano_linear[i,2])/(fano_hawkes[i,2] + fano_linear[i,2])
end
plt = plot(xlabel = "h (mV)", ylabel="Relative error", fmt = :png, legend=:topright)
plot!(plt, h_arr, y_e, label="Excite", color="blue")
plot!(plt, h_arr, y_i, label="Inhibit", color="red")
#n_spikes = 500_000

In [ ]:
y_e = zeros(num_h)
y_i = zeros(num_h)
for i in 2:num_h
    y_e[i] = abs(fano_hawkes[i,1] - fano_linear[i,1])
    y_i[i] = abs(fano_hawkes[i,2] - fano_linear[i,2])
end
plt = plot(xlabel = "h (mV)", ylabel="absolute error", fmt = :png, legend=:topleft)
plot!(plt, h_arr, y_e, label="Excite", color="blue")
plot!(plt, h_arr, y_i, label="Inhibit", color="red")

#n_spikes = 500_000

In [ ]:
length = size(h_arr,1)
# plt = plot(xlabel="h (mV)", ylabel="Voltage (mV)", fmt = :png, legend=:bottomright)
# plot!(plt, h_arr[1:length] , v_ssn[1:length,1], label = "V_Excite", color="blue")
# plot!(plt, h_arr[1:length] , v_ssn[1:length,2], label = "V_Inhibit", color="red")

plt = plot(xlabel="h (mV)", ylabel="E Rate (Hz)", fmt = :png, legend=:bottomright)
plot!(plt, h_arr[1:length] , rates_ssn[1:length,1], label = "R_Linear", color="blue")
plot!(plt, h_arr[1:length] , rates_hawkes_linear[1:length,1], label = "R_Hawkes", color="dark blue", linestyle=:dash)

In [ ]:
plt = plot(xlabel="h (mV)", ylabel="I Rate (Hz)", fmt = :png, legend=:bottomright)
plot!(plt, h_arr[1:length] , rates_ssn[1:length,2], label = "R_Linear",color="red")
plot!(plt, h_arr[1:length] , rates_hawkes_linear[1:length,2], label = "R_Hawkes",color="dark red", linestyle=:dash)

In [ ]:
y_e = zeros(num_h)
y_i = zeros(num_h)
for i in 2:num_h
    y_e[i] = 2*abs(rates_hawkes_linear[i,1] - rates_ssn[i,1])/(rates_hawkes_linear[i,1] + rates_ssn[i,1])
    y_i[i] = abs(rates_hawkes_linear[i,2] - rates_ssn[i,2])/(rates_hawkes_linear[i,2] + rates_ssn[i,2])
end
plt = plot(xlabel = "h (mV)", ylabel="Relative error", fmt = :png, legend=:topright)
plot!(plt, h_arr, y_e, label="rateE", color="blue")
plot!(plt, h_arr, y_i, label="rateI", color="red")
#n_spikes = 500_000

In [ ]:
length = size(h_arr,1)

plt = plot(xlabel="h (mV)", ylabel="Rate (Hz)", fmt = :png, legend=:bottomright)
plot!(plt, h_arr[1:length] , rates_ssn[1:length,2], label = "R_Linear_Inhibit",color="red")
plot!(plt, h_arr[1:length] , rates_hawkes_linear[1:length,2], label = "R_Hawkes_Inhibit",color="dark red")

In [ ]:
y_e = zeros(num_h)
y_i = zeros(num_h)
for i in 2:num_h
    y_e[i] = abs(rates_hawkes_linear[i,1] - rates_ssn[i,1])
    y_i[i] = abs(rates_hawkes_linear[i,2] - rates_ssn[i,2])
end
plt = plot(xlabel = "h (mV)", ylabel="absolute error", fmt = :png, legend=:topleft)
plot!(plt, h_arr, y_e, label="rateE", color="blue")
plot!(plt, h_arr, y_i, label="rateI", color="red")

#n_spikes = 500_000

In [ ]:
rates_hawkes_weights = rates_hawkes_linear
covE_h_weights = covE_h
covI_h_weights = covI_h
covEI_h_weights = covEI_h
println()

In [ ]:
rates_linear_weights_covdiag = rates_ssn
covE_l_weights_covdiag = covE_l
covI_l_weights_covdiag = covI_l
covEI_l_weights_covdiag = covEI_l
println()

In [ ]:
plt = plot(xlabel = "h (mV)", ylabel="covEE", fmt=:png, legend=:bottomright)
plot!(h_arr, covE_h_weights, label="covE_hawkes";color="blue")
# plot!(h_arr, rates_hawkes_weights[:,1], label="rates_hawkes_weights";color="black")

# plot!(h_arr, rates_linear_weights_covdiag[:,1], label="rates_linear_weights_covdiag";color="blue")
plot!(h_arr, covE_l_weights_covdiag, label="covE_linear";color="purple")

In [ ]:
plt = plot(xlabel = "h (mV)", ylabel="E mean", fmt=:png, legend=:bottomright)
plot!(h_arr, rates_hawkes_weights[:,1], label="rates_hawkes_weights";color="black")

plot!(h_arr, rates_linear_weights_covdiag[:,1], label="rates_linear_weights_covdiag";color="blue")

In [ ]:
plt = plot(xlabel = "h (mV)", ylabel="covEE", fmt=:png, legend=:bottomright)
plot!(h_arr, covE_h_weights, label="hawkes";color="blue")
plot!(h_arr, covE_l_weights_covdiag, label="linear";color="purple")

In [ ]:
plt = plot(xlabel = "h (mV)", ylabel="covII", fmt=:png, legend=:bottomright)
plot!(h_arr, covI_h_weights, label="hawkes";color="red")
# plot!(h_arr, rates_hawkes_weights[:,2], label="rates_hawkes_weights";color="black")

# plot!(h_arr, rates_linear_weights_covdiag[:,2], label="rates_linear_weights_covdiag";color="blue")
plot!(h_arr, covI_l_weights_covdiag, label="linear";color="black")

In [ ]:
plt = plot(xlabel = "h (mV)", ylabel="covEI", fmt=:png, legend=:topleft)
plot!(h_arr, covEI_h_weights, label="hawkes";color="black")
plot!(h_arr, covEI_l_weights_covdiag, label="linear";color="purple")

In [ ]:
y_e = zeros(num_h)
y_i = zeros(num_h)
for i in 2:num_h
    y_e[i] = 2*abs(covE_h_weights[i] - covE_l_weights_covdiag[i])/(covE_h_weights[i] + covE_l_weights_covdiag[i])
    y_i[i] = 2*abs(covI_h_weights[i] - covI_l_weights_covdiag[i])/(covI_h_weights[i] + covI_l_weights_covdiag[i])
end
plt = plot(xlabel = "h (mV)", ylabel="relative error", fmt = :png, legend=:topleft)
plot!(plt, h_arr, y_e, label="covEE", color="blue")
plot!(plt, h_arr, y_i, label="covII", color="red")

In [ ]:
y_e = zeros(num_h)
y_i = zeros(num_h)
for i in 2:num_h
    y_e[i] = abs(covE_h_weights[i] - covE_l_weights_covdiag[i])
    y_i[i] = abs(covI_h_weights[i] - covI_l_weights_covdiag[i])
end
plt = plot(xlabel = "h (mV)", ylabel="absolute error", fmt = :png, legend=:topleft)
plot!(plt, h_arr, y_e, label="covEE", color="red")
plot!(plt, h_arr, y_i, label="covII", color="blue")

In [ ]:
y_e = zeros(num_h)
y_i = zeros(num_h)
for i in 2:num_h
    y_e[i] = 2*abs(covEI_h_weights[i] - covEI_l_weights_covdiag[i])/abs(covEI_h_weights[i] + covEI_l_weights_covdiag[i])
end
plt = plot(xlabel = "h (mV)", ylabel="relative error", fmt = :png, legend=:topleft)
plot!(plt, h_arr, y_e, label="covEI", color="purple")

In [ ]:
y_e = zeros(num_h)
y_i = zeros(num_h)
for i in 2:num_h
    y_e[i] = abs(covEI_h_weights[i] - covEI_l_weights_covdiag[i])
end
plt = plot(xlabel = "h (mV)", ylabel="absolute error", fmt = :png, legend=:topleft)
plot!(plt, h_arr, y_e, label="covEI", color="purple")

In [ ]:
y_e = zeros(num_h)
y_i = zeros(num_h)
for i in 2:num_h
    y_e[i] = 2*abs(covE_h_weights[i] - covE_l_weights_covdiag[i])/(covE_h_weights[i] + covE_l_weights_covdiag[i])
    y_i[i] = 2*abs(covI_h_weights[i] - covI_l_weights_covdiag[i])/(covI_h_weights[i] + covI_l_weights_covdiag[i])
end
plt = plot(xlabel = "h", ylabel="relative diff with hawkes", fmt = :png, legend=:topleft)
plot!(plt, h_arr, y_e, label="covE")
plot!(plt, h_arr, y_i, label="covI")

In [ ]:
plot(h_arr, [rates_hawkes_weights[:,2] rates_linear_weights_covdiag[:,2]], label=["rates_hawkes_weights" "rates_linear_weights_covdiag"], xlabel = "h (mV)", ylabel="I mean", fmt=:png, legend=:bottomright)

In [ ]:
plot(h_arr, [covI_h_weights covI_l_weights_covdiag], label=["covI_hawkes_weights" "covI_l_weights_covdiag"], xlabel = "h (mV)", ylabel="I cov", fmt=:png, legend=:bottomright)

In [ ]:
rates_linear_noweights_covnondiag = rates_ssn
covE_l_noweights_covnondiag = covE_l
covI_l_noweights_covnondiag = covI_l
covEI_l_noweights_covnondiag = covEI_l
println()

In [ ]:
rates_linear_noweights_covdiag = rates_ssn
covE_l_noweights_covdiag = covE_l
covI_l_noweights_covdiag = covI_l
covEI_l_noweights_covdiag = covEI_l
println()

In [ ]:
rates_hawkes_noweights = rates_hawkes_linear
covE_h_noweights = covE_h
covI_h_noweights = covI_h
covEI_h_noweights = covEI_h
println()

In [ ]:
y_e = zeros(num_h)
y_i = zeros(num_h)
for i in 2:num_h
    y_e[i] = 2*abs(covEI_h_noweights[i] - covEI_l_noweights_covdiag[i])/(covEI_h_noweights[i] + covEI_l_noweights_covdiag[i])
    y_i[i] = 2*abs(covEI_h_noweights[i] - covEI_l_noweights_covnondiag[i])/(covEI_h_noweights[i] + covEI_l_noweights_covnondiag[i])
end
plt = plot(xlabel = "h", ylabel="relative diff of hawkes (covEI)", fmt = :png, legend=:topright)
plot!(plt, h_arr, y_e, label="with uncorrelated noise")
plot!(plt, h_arr, y_i, label="with correlated noise")

In [ ]:
y_e = zeros(num_h)
y_i = zeros(num_h)
for i in 2:num_h
    y_e[i] = 2*abs(covI_h_noweights[i] - covI_l_noweights_covdiag[i])/(covI_h_noweights[i] + covI_l_noweights_covdiag[i])
    y_i[i] = 2*abs(covI_h_noweights[i] - covI_l_noweights_covnondiag[i])/(covI_h_noweights[i] + covI_l_noweights_covnondiag[i])
end
plt = plot(xlabel = "h", ylabel="relative diff of hawkes (covI)", fmt = :png, legend=:topleft)
plot!(plt, h_arr, y_e, label="with uncorrelated noise")
plot!(plt, h_arr, y_i, label="with correlated noise")

In [ ]:
y_e = zeros(num_h)
y_i = zeros(num_h)
for i in 2:num_h
    y_e[i] = 2*abs(covE_h_noweights[i] - covE_l_noweights_covdiag[i])/(covEI_h_noweights[i] + covE_l_noweights_covdiag[i])
    y_i[i] = 2*abs(covE_h_noweights[i] - covE_l_noweights_covnondiag[i])/(covEI_h_noweights[i] + covE_l_noweights_covnondiag[i])
end
plt = plot(xlabel = "h", ylabel="relative diff of hawkes (covE)", fmt = :png, legend=:topleft)
plot!(plt, h_arr, y_e, label="with uncorrelated noise")
plot!(plt, h_arr, y_i, label="with correlated noise")

In [ ]:
plt = plot(xlabel = "h (mV)", ylabel="E cov, mean", fmt=:png, legend=:bottomright)
plot!(h_arr, covE_h_noweights, label="covE_hawkes_noweights";color="red")
plot!(h_arr, covE_h_weights, label="covE_hawkes_weights";color="dark red")
plot!(h_arr, rates_hawkes_weights[:,1], label="rates_hawkes_weights";color="black")

plot!(h_arr, rates_linear_noweights_covnondiag[:,1], label="rates_linear_noweights_covnondiag";color="blue")
plot!(h_arr, covE_l_noweights_covnondiag, label="covE_l_noweights_covnondiag";color="purple")

In [ ]:
plt = plot(xlabel = "h (mV)", ylabel="I cov, mean", fmt=:png, legend=:bottomright)
plot!(h_arr, covI_h_noweights, label="covI_hawkes_noweights";color="red")
plot!(h_arr, covI_h_weights, label="covI_hawkes_weights";color="dark red")
plot!(h_arr, rates_hawkes_weights[:,2], label="rates_hawkes_weights";color="black")

plot!(h_arr, rates_linear_noweights_covnondiag[:,2], label="rates_linear_noweights_covnondiag";color="blue")
plot!(h_arr, covI_l_noweights_covnondiag, label="covI_l_noweights_covnondiag";color="purple")

In [ ]:
plt = plot(xlabel = "h (mV)", ylabel="EI cov", fmt=:png, legend=:topleft)
plot!(h_arr, covEI_h_noweights, label="covEI_hawkes_noweights";color="red")
plot!(h_arr, covEI_h_weights, label="covEI_hawkes_weights";color="dark red")

plot!(h_arr, covEI_l_noweights_covnondiag, label="covEI_l_noweights_covnondiag";color="black")

In [ ]:
plt = plot(xlabel = "h (mV)", ylabel="E cov, mean", fmt=:png, legend=:bottomright)
plot!(h_arr, covE_h_noweights, label="covE_hawkes_noweights";color="red")
plot!(h_arr, covE_h_weights, label="covE_hawkes_weights";color="dark red")
plot!(h_arr, rates_hawkes_weights[:,1], label="rates_hawkes_weights";color="black")

plot!(h_arr, rates_linear_noweights_covdiag[:,1], label="rates_linear_noweights_covdiag";color="blue")
plot!(h_arr, covE_l_noweights_covdiag, label="covE_l_noweights_covdiag";color="purple")

In [ ]:
plt = plot(xlabel = "h (mV)", ylabel="I cov, mean", fmt=:png, legend=:bottomright)
plot!(h_arr, covI_h_noweights, label="covI_hawkes_noweights";color="red")
plot!(h_arr, covI_h_weights, label="covI_hawkes_weights";color="dark red")
plot!(h_arr, rates_hawkes_weights[:,2], label="rates_hawkes_weights";color="black")

plot!(h_arr, rates_linear_noweights_covdiag[:,2], label="rates_linear_noweights_covdiag";color="blue")
plot!(h_arr, covI_l_noweights_covdiag, label="covI_l_noweights_covdiag";color="purple")

In [ ]:
plt = plot(xlabel = "h (mV)", ylabel="EI cov", fmt=:png, legend=:topleft)
plot!(h_arr, covEI_h_noweights, label="covEI_hawkes_noweights";color="red")
plot!(h_arr, covEI_h_weights, label="covEI_hawkes_weights";color="dark red")

plot!(h_arr, covEI_l_noweights_covdiag, label="covEI_l_noweights_covdiag";color="purple")
plot!(h_arr, covEI_l_noweights_covnondiag, label="covEI_l_noweights_covnondiag";color="black")

In [ ]:
rates_linear_weights_covW = rates_ssn
covE_l_weights_covW = covE_l
covI_l_weights_covW = covI_l
covEI_l_weights_covW = covEI_l
println()

In [ ]:
rates_linear_weights_covnoW = rates_ssn
covE_l_weights_covnoW = covE_l
covI_l_weights_covnoW = covI_l
covEI_l_weights_covnoW = covEI_l
println()

In [ ]:
plt = plot(xlabel="h", ylabel="EI cov, mean", fmt = :png, legend=:bottomright)
plot!(h_arr,covEI_h_noweights, label="covEI_now"; color="red")
plot!(h_arr, covEI_l_noweights_covnoW, label="covEI_l_now";color="dark red")

In [ ]:
plt = plot(xlabel="h", ylabel="E cov, mean", fmt = :png, legend=:bottomright)
plot!(h_arr, rates_hawkes_noweights[:,1], label="rateE_h_now"; color="blue")
plot!(h_arr,covE_h_noweights, label="covE_now"; color="red")
plot!(h_arr, covE_l_noweights_covnoW, label="covE_l_now";color="dark red")
plot!(h_arr, rates_linear_noweights_covnoW[:,1], label="rates_l_now"; color="dark blue")

In [ ]:
plt = plot(xlabel="h", ylabel="I cov, mean", fmt = :png, legend=:bottomright)
plot!(h_arr, rates_hawkes_noweights[:,2], label="rateI_h_now"; color="blue")
plot!(h_arr,covI_h_noweights, label="covI_now"; color="red")
plot!(h_arr, covI_h_weights, label="covI_w";color="pink")
plot!(h_arr, covI_l_noweights_covnoW, label="covI_l_now";color="dark red")
plot!(h_arr, rates_linear_noweights_covnoW[:,2], label="rates_l_now"; color="dark blue")

In [ ]:
plt = plot(xlabel="h", ylabel="I cov, mean", fmt = :png, legend=:bottomright)
plot!(h_arr, rates_hawkes_noweights[:,2], label="rateI_h_now"; color="blue")
plot!(h_arr,covI_h_noweights, label="covI_now"; color="red")
plot!(h_arr, covI_l_noweights_covnoW, label="covI_l_now";color="dark red")
plot!(h_arr, rates_linear_noweights_covnoW[:,2], label="rates_l_now"; color="dark blue")

In [ ]:
plt = plot(xlabel="h", ylabel="covEI", fmt = :png, legend=:bottomright)
plot!(h_arr,covEI_h_weights, label="covEI_h_w"; color="red")
plot!(h_arr, covEI_l_weights_covnoW, label="covEI_l_now";color="dark red")
plot!(h_arr, covEI_l_weights_covW, label="covEI_l_w"; color="black")

In [ ]:
plt = plot(xlabel="h", ylabel="I cov, mean", fmt = :png, legend=:bottomright)
plot!(h_arr, rates_hawkes_weights[:,2], label="rateI_h_ w"; color="blue")
plot!(h_arr,covI_h_weights, label="covI_ w"; color="red")
plot!(h_arr, covI_l_weights_covnoW, label="covI_l_now";color="dark red")
plot!(h_arr, rates_linear_weights_covnoW[:,2], label="rates_l_now"; color="dark blue")
plot!(h_arr, covI_l_weights_covW, label="covI_l_w"; color="black")
plot!(h_arr, rates_linear_weights_covW[:,2], label="rates_l_w"; color="purple")

In [ ]:
plt = plot(xlabel="h", ylabel="I mean", fmt = :png, legend=:bottomright)
plot!(h_arr, rates_hawkes_weights[:,2], label="rateE_h_ w"; color="blue")
plot!(h_arr, rates_linear_weights_covnoW[:,2], label="rates_l_now"; color="dark blue")
plot!(h_arr, rates_linear_weights_covW[:,2], label="rates_l_w"; color="purple")

In [ ]:
plt = plot(xlabel="h", ylabel="I cov", fmt = :png, legend=:bottomright)
plot!(h_arr,covI_h_weights, label="covI_h_ w"; color="red")
plot!(h_arr, covI_l_weights_covnoW, label="covI_l_now";color="dark red")
plot!(h_arr, covI_l_weights_covW, label="covI_l_w"; color="black")

In [ ]:
plt = plot(xlabel="h", ylabel="E cov, mean", fmt = :png, legend=:bottomright)
plot!(h_arr, rates_hawkes_weights[:,1], label="rateE_ w"; color="blue")
plot!(h_arr,covE_h_weights, label="covE_ w"; color="red")
plot!(h_arr, covE_l_weights_covnoW, label="covE_l_now";color="dark red")
plot!(h_arr, rates_linear_weights_covnoW[:,1], label="rates_l_now"; color="dark blue")
plot!(h_arr, covE_l_weights_covW, label="covE_l_w"; color="black")
plot!(h_arr, rates_linear_weights_covW[:,1], label="rates_l_w"; color="purple")

In [ ]:
plt = plot(xlabel="h", ylabel="E mean", fmt = :png, legend=:bottomright)
plot!(h_arr, rates_hawkes_weights[:,1], label="rateE_h_ w"; color="blue")
plot!(h_arr, rates_linear_weights_covnoW[:,1], label="rates_l_now"; color="dark blue")
plot!(h_arr, rates_linear_weights_covW[:,1], label="rates_l_w"; color="purple")

In [ ]:
plt = plot(xlabel="h", ylabel="E cov", fmt = :png, legend=:bottomright)
plot!(h_arr,covE_h_weights, label="covE_h_ w"; color="red")
plot!(h_arr, covE_l_weights_covnoW, label="covE_l_now";color="dark red")
plot!(h_arr, covE_l_weights_covW, label="covE_l_w"; color="black")

In [ ]:
plt = plot(xlabel="mean", ylabel="cov", fmt = :png, legend=:bottomright)
plot!(rates_hawkes_noweights[:,1] ,covE_h_noweights, label="E_no w")
plot!(rates_hawkes_weights[:,1] ,covE_h_weights, label="E_ w")
plot!(rates_hawkes_weights[:,1], rates_hawkes_weights[:,1], linestyle=:dash)

In [ ]:
plt = plot(xlabel="mean", ylabel="cov", fmt = :png, legend=:bottomright)
plot!(rates_hawkes_noweights[:,2] ,covI_h_noweights, label="I_no w")
plot!(rates_hawkes_weights[:,2] ,covI_h_weights, label="I_ w")
plot!(rates_hawkes_weights[:,2], rates_hawkes_weights[:,2], linestyle=:dash)

In [ ]:
plt = plot(xlabel="h", ylabel="cov, mean", fmt = :png, legend=:bottomright)
plot!(h_arr, rates_hawkes_noweights[:,1], label="rateE_no w")
plot!(h_arr ,covE_h_noweights, label="covE_no w")
plot!(h_arr, rates_hawkes_weights[:,1], label="rateE_ w")
plot!(h_arr,covE_h_weights, label="covE_ w")

In [ ]:
plt = plot(xlabel="h", ylabel="cov, mean", fmt = :png, legend=:bottomright)
plot!(h_arr, rates_hawkes_noweights[:,2], label="rateI_no w")
plot!(h_arr ,covI_h_noweights, label="covI_no w")
plot!(h_arr, rates_hawkes_weights[:,2], label="rateI_ w")
plot!(h_arr,covI_h_weights, label="covI_ w")

In [ ]:
plt = plot(xlabel="h", ylabel="cov, mean", fmt = :png, legend=:topleft)
plot!(h_arr ,covEI_h_noweights, label="covEI_no w")
plot!(h_arr,covEI_h_weights, label="covEI_ w")

In [ ]:
covEI_l_abs = abs.(covEI_l)
plt1 = plot(h_arr, [covE_h covE_l], label=["hawkes" "linear"],xlabel="h (mV)", ylabel="covE", fmt = :png, legend=:bottomright)
plt2 = plot(h_arr, [covI_h covI_l], label=["hawkes" "linear"], xlabel="h (mV)", ylabel="covI", fmt = :png, legend=:bottomright)
plt3 = plot(h_arr, [covEI_h covEI_l], label=["hawkes" "linear"], xlabel="h (mV)", ylabel="covEI", fmt = :png, legend=:bottomright)
plt4 = plot(h_arr, [covEI_h covEI_l_abs], label=["hawkes" "linear"], xlabel="h (mV)", ylabel=" abs covEI", fmt = :png, legend=:bottomright)
plot(plt1, plt2, plt3, plt4)

In [ ]:
cov_l = [covE_l covI_l covEI_l]
cov_h = [covE_h covI_h covEI_h]
plt = plot(xlabel="cov_l", ylabel="cov_h", fmt = :png, legend=false)
plot!(cov_l, cov_h, seriestype = :scatter)
plot!(plt, cov_l , cov_l)

In [ ]:
rates_ssn = zeros(num_h,2)
covE_l = zeros(num_h)
covI_l = zeros(num_h)
covEI_l = zeros(num_h)

cov_noise = [sigma_E^2 rho*sigma_E^2
            rho*sigma_I^2 sigma_I^2]
run_ssn_multiple_h!(num_h, h_arr, cov_noise, rates_ssn, covE_l, covI_l, covEI_l)

In [ ]:
covEI_l_abs = abs.(covEI_l)
plt5 = plot(h_arr, [covE_h covE_l], label=["hawkes" "linear"],xlabel="h (mV)", ylabel="covE", fmt = :png, legend=:bottomright)
plt6 = plot(h_arr, [covI_h covI_l], label=["hawkes" "linear"], xlabel="h (mV)", ylabel="covI", fmt = :png, legend=:bottomright)
plt7 = plot(h_arr, [covEI_h covEI_l], label=["hawkes" "linear"], xlabel="h (mV)", ylabel="covEI", fmt = :png, legend=:bottomright)
plt8 = plot(h_arr, [covEI_h covEI_l_abs], label=["hawkes" "linear"], xlabel="h (mV)", ylabel=" abs covEI", fmt = :png, legend=:bottomright)
plot(plt5, plt6, plt7, plt8)

In [ ]:
cov_l = [covE_l covI_l covEI_l]
cov_h = [covE_h covI_h covEI_h]
plt = plot(xlabel="cov_l", ylabel="cov_h", fmt = :png, legend=false)
plot!(cov_l, cov_h, seriestype = :scatter)
plot!(plt, cov_l , cov_l)

In [ ]:
plt = plot(xlabel="h (mV)", ylabel="E Rate", fmt = :png, legend=:bottomright)
plot!(plt, h_arr, rates_ssn[:,1], label = "linear")
plot!(plt, h_arr, rates_hawkes_linear[:,1], label = "hawkes")

In [ ]:
plt = plot(xlabel="h (mV)", ylabel="I Rate", fmt = :png, legend=:bottomright)
plot!(plt, h_arr, rates_ssn[:,2], label = "linear")
plot!(plt, h_arr, rates_hawkes_linear[:,2], label = "hawkes")

In [ ]:
plt = plot(xlabel="h (mV)", ylabel="covE", fmt = :png, legend=:bottomright)
plot!(plt, h_arr , covE_h, label="hawkes")
plot!(plt, h_arr , covE_l, label="linear")

In [ ]:
plt = plot(xlabel="h (mV)", ylabel="covI", fmt = :png, legend=:bottomright)
plot!(plt, h_arr , covI_h, label="hawkes")
plot!(plt, h_arr , covI_l, label="linear")

In [ ]:
plt = plot(xlabel="h (mV)", ylabel="covEI", fmt = :png, legend=:bottomright)
plot!(plt, h_arr , covEI_h, label="hawkes")
plot!(plt, h_arr , covEI_l, label="linear")

In [ ]:
covEI_l_abs = abs.(covEI_l)
plt = plot(xlabel="h (mV)", ylabel="covEI", fmt = :png, legend=:bottomright)
plot!(plt, h_arr , covEI_h, label="hawkes")
plot!(plt, h_arr , covEI_l_abs, label="linear")

In [ ]:
plt = plot(xlabel="h", ylabel="Rate")
plot!(plt, h_arr , rates_ssn[:,1], label = "rates_ssnE")
plot!(plt, h_arr , rates_ssn[:,2], label = "rates_ssnI")

In [ ]:
plt = plot(xlabel="h", ylabel="Rate")
plot!(plt, h_arr , rates_hawkes_linear[:,1], label = "rates_hawkes_linearE")
plot!(plt, h_arr , rates_hawkes_linear[:,2], label = "rates_hawkes_linearI")